### For each sequence in the test set, record the genome origin 
### To see the classifier performance across different genomes (See EukRep paper, Fig 2C)

In [ ]:
import os
import pandas as pd
import glob
from Bio import SeqIO
import re

In [ ]:
os.chdir("/fs/scratch/PAS0439/Ming/databases/gut_eukaryotes_classifier/test/")

In [ ]:
fungi = glob.glob("fungi_5kb/*")
fungi_genome_seq = {}
for genome in fungi:
    ID = ''.join(genome.split('/')[1].split("_5000bp"))
    seqlist = []
    records = SeqIO.parse(genome, "fasta")
    for record in records:
        seqlist.append(record.id)
    fungi_genome_seq[ID] = seqlist

In [ ]:
proka = glob.glob("proka_5kb/*")
proka_genome_seq = {}
for genome in proka:
    ID = genome.split('/')[1].split("_genomic_5000bp.fasta")[0]
    seqlist = []
    records = SeqIO.parse(genome, "fasta")
    for record in records:
        seqlist.append(record.id)
    proka_genome_seq[ID] = seqlist

In [ ]:
proto = glob.glob("protozoa_5kb/*")
proto_genome_seq = {}
for genome in proto:
    ID = re.search("(GCF_[0-9]*\.[0-9]{1}).*", genome).group(1)
    seqlist = []
    records = SeqIO.parse(genome, "fasta")
    for record in records:
        seqlist.append(record.id)
    fungi_genome_seq[ID] = seqlist

In [ ]:
sags_genome_seq = {}
seqlist = []
records = SeqIO.parse("sags_test_5000bp.fasta", "fasta")
for record in records:
    genome = str(record.id).split('_')[0]
    if genome in sags_genome_seq:
        sags_genome_seq[genome].append(record.id)
    else:
        sags_genome_seq[genome] = [record.id]
        

In [ ]:
genome_seq_id = {**sags_genome_seq, **proto_genome_seq, **fungi_genome_seq, **proka_genome_seq}

In [ ]:
genomes_list = [] 
seqs_list= []

for genome, seqs in genome_seq_id.items():
    for seq in seqs:
        genomes_list.append(genome)
        seqs_list.append(seq)
    
genome_seq_df = pd.DataFrame.from_dict({"genome": genomes_list, "seq": seqs_list})

In [ ]:
protozoa = pd.read_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/dataset_protozoa_repre.csv")
proka = pd.read_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/dataset_proka_repre.csv")
fungi = pd.read_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/dataset_fungi_repre.csv")
sags = pd.read_csv("sags_test_5000bp.csv", header = None)

In [ ]:
category = [] 
for index, row in genome_seq_df.iterrows():
    genome = row['genome']
    if genome in list(protozoa.id):
        category.append("protozoa")
    elif genome in list(proka.id):
        category.append("prokaryotes")
    elif genome in list(fungi.files):
        category.append("fungi")
    else:
        category.append("SAGs")
    
    

In [ ]:
genome_seq_df["category"] = category

In [ ]:
genome_seq_df.to_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/testset_seq_origin.csv", index = None)